<a href="https://colab.research.google.com/github/JCaballerot/Recommender-Systems/blob/main/Autoencoder_Recommender/Autoencoder_Collaborative_Filtering_Last_fm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> ALS Collaborative Filtering - Last.fm </font></h1>

---

**Índice**

- 1. Introducción
- 2. Carga y Filtrado de Datos
- 3. Creación del Modelo Autoencoder Convolucional
- 4. Generación de Recomendaciones
- 5. Validación
- 6. Conclusiones


## 1. Introducción

En este laboratorio, exploraremos el uso de autoencoders convolucionales para generar recomendaciones en el ámbito de las interacciones entre usuarios y artistas en Last.fm. Los autoencoders son modelos de aprendizaje no supervisado que aprenden a reconstruir su entrada pasando por una representación latente más compacta, capturando así las características esenciales de los datos. Al incorporar capas convolucionales, el modelo puede detectar patrones locales y estructuras complejas en los datos, lo que es especialmente útil en conjuntos de datos con relaciones no lineales y distribuciones heterogéneas.



Instalamos las librerías necesarias.



In [ ]:
# Instalación de librerías necesarias
!pip install tensorflow

# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models


## 2. Carga y Filtrado de Datos

Cargamos el dataset y aplicamos un filtro "long tail" para mejorar la calidad del análisis, manteniendo solo los artistas con al menos 50 escuchas. Este enfoque reduce el impacto de artistas menos populares y permite centrarse en recomendaciones más relevantes.

In [ ]:
# Descargar el dataset de Last.fm desde Kaggle
!pip install kaggle

from google.colab import files
files.upload()  # Sube tu archivo kaggle.json aquí

# Configurar Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descargar y descomprimir el dataset de Last.fm
!kaggle datasets download -d japarra27/lastfm-dataset
!unzip lastfm-dataset.zip


In [ ]:
# Cargar el dataset
data = pd.read_parquet("lastfm_union.parquet")[:10_000_000]


In [ ]:
data.head()

**Filtrado "long tail"**


In [ ]:
# Contar las escuchas por artista
artist_listen_counts = data.groupby('artist_name').size().sort_values(ascending=False)

# Visualizar distribución long tail
plt.figure(figsize=(12, 6))
plt.bar(range(len(artist_listen_counts)), artist_listen_counts, color='lightblue')
plt.title('Distribución del Número de Escuchas por Artista (Long Tail)')
plt.xlabel('Artistas ordenados por popularidad')
plt.ylabel('Número de escuchas')
plt.ylim(1, 4000)
plt.show()


In [ ]:
artist_listen_counts

In [ ]:
# Filtrar artistas con al menos 100 escuchas
min_listens_per_artist = 100
popular_artists = artist_listen_counts[artist_listen_counts >= min_listens_per_artist].index
data_filtered = data[data['artist_name'].isin(popular_artists)]


In [ ]:
# Filtrar usuarios con al menos 100 escuchas
users_listen_counts = data_filtered.groupby('user_id').size().sort_values(ascending=False)
users_listen_counts

In [ ]:
min_listens_per_user = 100
popular_users = users_listen_counts[users_listen_counts >= min_listens_per_user].index
data_filtered = data_filtered[data_filtered['user_id'].isin(popular_users)]


In [ ]:
data_filtered.groupby('user_id').size().sort_values(ascending=False).tail()

In [ ]:
data_filtered.groupby('artist_name').size().sort_values(ascending=False).tail()

## 3. Muestreo de datos

Ahora que tenemos los datos filtrados, procederemos a entrenar un modelo de recomendación usando ALS con la biblioteca surprise. En surprise, podemos utilizar el algoritmo BaselineOnly con el método de estimación configurado como ALS.

Primero, preparamos los datos en el formato que requiere surprise.

In [ ]:
# Crear el DataFrame con el recuento de escuchas
user_artist_df = data_filtered.groupby(['user_id', 'artist_name']).size().reset_index(name='listens')
user_artist_df

In [ ]:
np.percentile(user_artist_df['listens'], 95)

In [ ]:
trainset_scaled = user_artist_df
trainset_scaled['listens'] = user_artist_df.listens/np.percentile(user_artist_df['listens'], 95)

**División del Conjunto de Datos**


In [ ]:
trainset_scaled

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba estratificando por usuario
train_data, test_data = sk_train_test_split(trainset_scaled,
    test_size = 0.2,
    random_state = 42,
    stratify = trainset_scaled['user_id']
)

## 4. Creación del Modelo Autoencoder Convolucional

Implementaremos un autoencoder convolucional. Los autoencoders son modelos que aprenden a reconstruir su entrada pasando por una representación latente más pequeña, capturando así las características más importantes de los datos. Al utilizar convoluciones, el modelo puede capturar patrones locales en las interacciones usuario-artista.

### 4.1 Preparación de los Datos para el Autoencoder

Primero, necesitamos preparar los datos en un formato adecuado para el autoencoder. Esto implica crear una matriz de interacción usuario-artista donde cada fila representa un usuario y cada columna representa un artista

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Copias de los conjuntos de datos para evitar advertencias
train_data_enc = train_data.copy()
test_data_enc = test_data.copy()

# Inicializar los codificadores
user_encoder = LabelEncoder()
artist_encoder = LabelEncoder()

# Ajustar y transformar los IDs de usuarios y artistas en train_data
train_data_enc['user_id_enc'] = user_encoder.fit_transform(train_data_enc['user_id'])
train_data_enc['artist_id_enc'] = artist_encoder.fit_transform(train_data_enc['artist_name'])

# Transformar los IDs de usuarios y artistas en test_data utilizando los mismos codificadores
test_data_enc['user_id_enc'] = user_encoder.transform(test_data_enc['user_id'])
test_data_enc['artist_id_enc'] = artist_encoder.transform(test_data_enc['artist_name'])


### 4.1.2 Creación de la Matriz de Interacción

Construimos la matriz de interacción utilizando los índices codificados.

In [ ]:
import numpy as np

# Obtener el número total de usuarios y artistas en train_data
num_users = train_data_enc['user_id_enc'].nunique()
num_artists = train_data_enc['artist_id_enc'].nunique()

# Crear la matriz de interacción vacía para train_data
interaction_matrix = np.zeros((num_users, num_artists))

# Rellenar la matriz con los valores de escuchas
for row in train_data_enc.itertuples():
    interaction_matrix[row.user_id_enc, row.artist_id_enc] = row.listens

print(f"Matriz de interacción de entrenamiento: {interaction_matrix.shape}")


### 4.1.3 Normalización de los Datos

Es recomendable normalizar los datos para que los valores estén entre 0 y 1, facilitando el entrenamiento del autoencoder.

In [ ]:
# Normalizar los valores entre 0 y 1
max_listens = interaction_matrix.max()
interaction_matrix_norm = interaction_matrix / max_listens


Estimating biases using als...


### 4.2 Construcción del Autoencoder Convolucional


Ahora construiremos el modelo del autoencoder convolucional utilizando TensorFlow y Keras.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Añadir una dimensión adicional para las convoluciones
interaction_matrix_reshaped = interaction_matrix_norm.reshape((interaction_matrix_norm.shape[0], interaction_matrix_norm.shape[1], 1))

# Definir la entrada del modelo
input_shape = (interaction_matrix_reshaped.shape[1], 1)
input_layer = layers.Input(shape=input_shape)

# Codificador
x = layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(input_layer)
x = layers.MaxPooling1D(pool_size=2, padding='same')(x)
x = layers.Conv1D(16, kernel_size=3, activation='relu', padding='same')(x)
encoded = layers.MaxPooling1D(pool_size=2, padding='same')(x)

# Decodificador
x = layers.Conv1D(16, kernel_size=3, activation='relu', padding='same')(encoded)
x = layers.UpSampling1D(size=2)(x)
x = layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(x)
x = layers.UpSampling1D(size=2)(x)

# Capa de salida
output_layer = layers.Conv1D(1, kernel_size=3, activation='sigmoid', padding='same')(x)

# Construir y compilar el modelo
autoencoder = models.Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

# Resumen del modelo
autoencoder.summary()


R² en el conjunto de entrenamiento: 0.08873116760890731


### 4.3 Entrenamiento del Modelo


Entrenamos el autoencoder utilizando únicamente train_data.

In [ ]:
# Entrenar el modelo con el conjunto de entrenamiento
history = autoencoder.fit(
    interaction_matrix_reshaped, interaction_matrix_reshaped,
    epochs=20,
    batch_size=64,
    shuffle=True
)


## 5. Generación de Recomendaciones



### 5.1 Reconstrucción de la Matriz de Interacción


Obtenemos las predicciones del modelo para todos los usuarios en train_data.



In [ ]:
# Reconstruir la matriz de interacción
reconstructed = autoencoder.predict(interaction_matrix_reshaped)

# Remover la dimensión adicional
reconstructed = reconstructed.reshape((interaction_matrix.shape[0], interaction_matrix.shape[1]))

# Desnormalizar las predicciones
reconstructed_denorm = reconstructed * max_listens
